In [50]:
#importing libraries
import pandas as pd
import sqlite3
from google.colab import files


In [51]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [52]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [53]:

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()


with open("restaurants.sql", "r") as file:
    sql_script = file.read()


cursor.executescript(sql_script)


restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [54]:
orders_users = orders.merge(
    users,
    on="user_id",
    how="left"
)

orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [55]:
final_df = orders_users.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [56]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [57]:
# final merged dataset
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
#Order Trends Over Time
order_trends = final_df.groupby("order_date")["order_id"].count()
order_trends


,order_id
order_date,
01-01-2023,29
01-01-2024,27
01-02-2023,25
01-03-2023,37
01-04-2023,32
...,...
31-05-2023,33
31-07-2023,27
31-08-2023,24


In [59]:
#User Behavior Patterns
user_behavior = final_df.groupby("user_id")["order_id"].count()
user_behavior


,order_id
user_id,
1,1
2,10
3,2
4,4
5,5
...,...
2996,3
2997,5
2998,6


In [60]:
#City-wise Performance
city_performance = final_df.groupby("city")["total_amount"].sum()
city_performance





,total_amount
city,
Bangalore,2206946.58
Chennai,1990513.03
Hyderabad,1889366.58
Pune,1924797.93


In [61]:
#Cuisine-wise Performance
cuisine_performance = final_df.groupby("cuisine")["total_amount"].sum()
cuisine_performance


,total_amount
cuisine,
Chinese,1930504.65
Indian,1971412.58
Italian,2024203.80
Mexican,2085503.09


In [62]:
#Membership Impact (Gold vs Regular)
membership_analysis = final_df.groupby("membership")["total_amount"].agg(
    total_revenue="sum",
    avg_order_value="mean",
    total_orders="count"
)
membership_analysis


,total_revenue,avg_order_value,total_orders
membership,,,
Gold,3975364.89,797.145556,4987
Regular,4036259.23,805.158434,5013


In [63]:
#Revenue Distribution & Seasonality
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["month"] = final_df["order_date"].dt.month

monthly_revenue = final_df.groupby("month")["total_amount"].sum()
monthly_revenue


/tmp/ipython-input-386404256.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


,total_amount
month,
1,663020.26
2,630867.40
3,716738.98
4,666097.18
5,668428.61
6,610822.93
7,688559.45
8,694987.58
9,653838.07


In [64]:
#Section 1
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [65]:
cuisine_aov = (
    final_df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_aov


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [66]:
high_value_users = (
    final_df.groupby("user_id")["total_amount"]
    .sum()
)

count_users = (high_value_users > 1000).sum()
count_users


np.int64(2544)

In [67]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

rating_revenue = (
    final_df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


/tmp/ipython-input-3954528916.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"]


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [68]:
gold_city_aov = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_aov


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [69]:
cuisine_stats = final_df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("restaurant_count")

cuisine_stats


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [70]:
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)
percentage


50

In [71]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
filtered.sort_values("avg_order_value", ascending=False).head()


,avg_order_value,total_orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [72]:
combo_revenue = (
    final_df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [73]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

quarter_revenue = (
    final_df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [74]:
mcq_restaurants = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

final_df[final_df["restaurant_name_x"].isin(mcq_restaurants)] \
    .groupby("restaurant_name_x") \
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    ) \
    .query("total_orders < 20") \
    .sort_values("avg_order_value", ascending=False)


,avg_order_value,total_orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [75]:
#section 2
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders_count



4987

In [76]:
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [77]:
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

In [78]:
gold_avg_order_value = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)


np.float64(797.15)

In [79]:
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [80]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city
gold_orders_top_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

gold_orders_top_city



1337

In [81]:
#section 3
final_df.shape[0]


10000